In [4]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow_datasets as tfds
dataset, dataset_info = tfds.load(name='malaria', shuffle_files=True, with_info=True, as_supervised=True, split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'])

In [5]:
# Preprocessing function
def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])  # Resize images to 224x224
    image = tf.cast(image, tf.float32)  # Convert images to float32
    return image, label

# Apply preprocessing
train_dataset, valid_dataset, test_dataset = dataset

train_dataset = train_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# Normalization function
def normalise(image, label):
    return image / 255, label

# Apply normalization
train_dataset = train_dataset.map(normalise)
valid_dataset = valid_dataset.map(normalise)
test_dataset = test_dataset.map(normalise)

# Apply shuffling, batching, and prefetching
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, InputLayer, RandomRotation, RandomFlip, Resizing, CenterCrop, BatchNormalization, Flatten, Dropout
from tensorflow.keras.models import Sequential
from keras_tuner import HyperModel
from tensorflow.keras.activation import LeakyReLU
class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()

        # Input Layer
        model.add(InputLayer(shape=(224, 224, 3)))

        # Data Augmentation Layers
        model.add(Resizing(224, 224))
        model.add(RandomRotation(0.2))
        model.add(RandomFlip('horizontal'))
        model.add(CenterCrop(200, 200))

        # Convolutional Layers with tunable number of filters and kernel size
        for i in range(4):
            model.add(Conv2D(
                filters=hp.Int(f'conv_{i}_filters', min_value=8, max_value=64, step=8),
                kernel_size=hp.Choice(f'conv_{i}_kernel_size', values=[3, 5]),
                padding='same',
                strides=2,
                activation=LeakyReLU(alpha=0.1)
            ))
            model.add(BatchNormalization())
        
        # Global Pooling Layer
        model.add(Flatten())

        # Fully Connected Layers with tunable units and dropout rate
        model.add(Dense(
            units=hp.Int('dense_1_units', min_value=64, max_value=256, step=64),
            activation=LeakyReLU(alpha=0.1)
        ))
        model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

        model.add(Dense(
            units=hp.Int('dense_2_units', min_value=64, max_value=512, step=64),
            activation=LeakyReLU(alpha=0.1)
        ))
        model.add(BatchNormalization())
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.5, step=0.1)))

        # Output Layer
        model.add(Dense(units=1, activation='sigmoid'))

        # Compile the model with a tunable learning rate
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model

In [32]:
from keras_tuner import RandomSearch

# Define the tuner
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=10,  # Number of different hyperparameter combinations to try
    executions_per_trial=2,  # Number of models to build and average
    directory='hyper_tuning',
    project_name='cnn_tuning'
)

In [33]:
from tensorflow.keras.callbacks import TensorBoard

# TensorBoard log directory
log_dir = './logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Start the hyperparameter search
tuner.search(
    train_dataset,
    validation_data=valid_dataset,
    epochs=10,
    callbacks=[tensorboard_callback]
)

Trial 10 Complete [00h 11m 49s]
val_accuracy: 0.9548258483409882

Best val_accuracy So Far: 0.9570029079914093
Total elapsed time: 03h 15m 36s


In [37]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The best hyperparameters are:
- Conv1 Filters: {best_hps.get('conv_0_filters')}
- Conv2 Filters: {best_hps.get('conv_1_filters')}
- Dense1 Units: {best_hps.get('dense_1_units')}
- Dense2 Units: {best_hps.get('dense_2_units')}
- Dropout1 Rate: {best_hps.get('dropout_1')}
- Dropout2 Rate: {best_hps.get('dropout_2')}
- Learning Rate: {best_hps.get('learning_rate')}
""")


The best hyperparameters are:
- Conv1 Filters: 16
- Conv2 Filters: 8
- Dense1 Units: 256
- Dense2 Units: 64
- Dropout1 Rate: 0.30000000000000004
- Dropout2 Rate: 0.4
- Learning Rate: 0.0039852321056818825



In [44]:
# Build the best model
best_model = tuner.hypermodel.build(best_hps)
log_dir = './logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch='100,132')

# Train the best model
history = best_model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=20,
    callbacks=[tensorboard_callback]
)

2024-09-07 15:35:04.290720: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2024-09-07 15:35:04.291079: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.
2024-09-07 15:35:04.294569: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:892] Profiler found 1 GPUs


Epoch 1/20


2024-09-07 15:35:04.949182: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2024-09-07 15:35:04.949663: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1036] CUPTI activity buffer flushed


101/689 ━━━━━━━━━━━━━━━━━━━━ 25s 44ms/step - accuracy: 0.5786 - loss: 0.8042

2024-09-07 15:35:13.917270: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:103] Profiler session initializing.
2024-09-07 15:35:13.917316: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:118] Profiler session started.


131/689 ━━━━━━━━━━━━━━━━━━━━ 25s 47ms/step - accuracy: 0.5862 - loss: 0.7828

2024-09-07 15:35:15.808486: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:68] Profiler session collecting data.
2024-09-07 15:35:15.813263: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1036] CUPTI activity buffer flushed
2024-09-07 15:35:16.015961: I external/local_xla/xla/backends/profiler/gpu/cupti_collector.cc:534]  GpuTracer has collected 34087 callback api events and 34169 activity events. 


134/689 ━━━━━━━━━━━━━━━━━━━━ 28s 51ms/step - accuracy: 0.5871 - loss: 0.7807

2024-09-07 15:35:16.230499: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:130] Profiler session tear down.
2024-09-07 15:35:16.243531: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:147] Collecting XSpace to repository: ./logs/20240907-115125/plugins/profile/2024_09_07_15_35_16/0f9d3fda392b.xplane.pb


689/689 ━━━━━━━━━━━━━━━━━━━━ 45s 59ms/step - accuracy: 0.7448 - loss: 0.5198 - val_accuracy: 0.9253 - val_loss: 0.2086
Epoch 2/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 53s 54ms/step - accuracy: 0.9297 - loss: 0.2086 - val_accuracy: 0.9503 - val_loss: 0.1606
Epoch 3/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 32s 46ms/step - accuracy: 0.9431 - loss: 0.1738 - val_accuracy: 0.9078 - val_loss: 0.2964
Epoch 4/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 33s 47ms/step - accuracy: 0.9470 - loss: 0.1722 - val_accuracy: 0.9507 - val_loss: 0.1944
Epoch 5/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - accuracy: 0.9500 - loss: 0.1593 - val_accuracy: 0.9539 - val_loss: 0.1649
Epoch 6/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 33s 47ms/step - accuracy: 0.9486 - loss: 0.1597 - val_accuracy: 0.9525 - val_loss: 0.1387
Epoch 7/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 32s 46ms/step - accuracy: 0.9475 - loss: 0.1618 - val_accuracy: 0.9358 - val_loss: 0.1995
Epoch 8/20
689/689 ━━━━━━━━━━━━━━━━━━━━ 32s 47ms/step - accuracy: 0.9503 - loss: 0.1535 - val_accurac

In [46]:
%reload_ext tensorboard

In [52]:
%tensorboard --logdir=./logs --bind_all

Reusing TensorBoard on port 6006 (pid 32179), started 0:33:04 ago. (Use '!kill 32179' to kill it.)